In [655]:
import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl 
import re
#from itertools import islice

In [656]:
# path to parent folder containing the folders with our data
directory = r'C:\Users\jrbrz\Desktop\690\assignments'

In [747]:
### loop through the FAA data to merge it to a master df

# the name of the folder the raw data files are in
faaFolder = '\\FAAdataOriginal'
#blank "master" df to append all files into for analysis
faa = pd.DataFrame(columns = ['Date','Event','City','State'])# + list(range(0,27,1)))

# looping through each raw file, cleaning up encoding issues 
# and appending it to the master df
for entry in os.scandir(directory + faaFolder):
    try:
        # creating the file path
        file = "FAAdataOriginal\\" + entry.name
        
        # loading the file into the openpyxl handler
        wb = load_workbook(file)
        
        # grabbing actual worksheet name in the xlsx file
        for s in range(len(wb.sheetnames)):
            sheet = wb.sheetnames[s]
            
        # activating the worksheet    
        ws = wb[sheet]
        
        # grabbing the data from the worksheet
        data = ws.values
        
        # grabbing the column header names from the file
        columns = next(data)[0:]
        
        # moving the data into a data frame
        df = pd.DataFrame(data, columns=columns).iloc[:,0:4]
        ### cleaning the Event text for encoding oddities
        # strips the newline characters at the start of the strings
        df['Event'] = df['Event'].apply(lambda x: x.strip())
        # removes the ASCII character that is showing up
        df['Event'] = df['Event'].apply(lambda x: x.replace("_x000D_",""))
        # removes double \n characters to make cleaner splitting below in the next step
        df['Event'] = df['Event'].apply(lambda x: x.replace("\n\n","\n"))
        df['Event'] = df['Event'].apply(lambda x: x.replace("\n\n\n","\n"))
        df['Event'] = df['Event'].apply(lambda x: x.replace("\n\n\n\n","\n"))
        
        # parsing out the Event text string into separate data fields
        ##temp = df['Event'].apply(lambda x: pd.Series(str(x).split("\n")))
        
        # joining the separated 
        ##df = df.join(temp)
        df['File'] = entry.name
        #added the df from each individual file into the master df
        faa = faa.append(df, ignore_index=True).fillna(np.NaN)
        
        
    except:  print(entry.name)        

faa.to_csv("faa_data2.csv")


# this section is to organize all of the similar categories into their own columns
# this splits the text strings based on the category list variable directly below
categories = [r'[UAS]*MOR Alert for ','Number:', 'Type:', 
              'Date/Time:','A/C:', 'Summary:', "FOLLOW UP INFO FROM FAA OPS:"]

for j in range(len(categories)): 

        if j == len(categories)-1:
            #print(categories[j])
            faa[categories[j]] = faa["Event"].apply(lambda x: re.split(categories[j],x)[1] if len(re.split(categories[j],x))>1 else np.NaN).replace("\n","")

        elif categories[j] == '[UAS]*MOR Alert for ':
            faa["PRELIM"] = faa["Event"].apply(#lambda x: re.split(categories[j], x)[0].replace('PRELIM INFO FROM FAA OPS:', "")).replace("\n","")
                lambda x: re.split(categories[j], x)[0].replace('PRELIM INFO FROM FAA OPS:', "") if re.search(r"^\s*UAS\s*", x) == None else np.NaN).replace("\n","")

        elif categories[j] == 'Summary:':
            faa[categories[j-1]] = faa["Event"].apply(lambda x: re.search(r''+categories[j-1]+r"(.*?)\n"+categories[j],x).group(1) if re.search(r''+categories[j-1]+r"(.*?)\n"+categories[j],x) else np.NaN).replace("\n","")
            faa[categories[j]] = faa["Event"].apply(lambda x: re.split(categories[j],x)[1] if len(re.split(categories[j],x))>1 else np.NaN).replace("\n","")
            
        else:
            faa[categories[j-1]] = faa["Event"].apply(lambda x: re.search(r''+categories[j-1]+r"(.*?)\n"+categories[j],x).group(1) if re.search(r''+categories[j-1]+r"(.*?)\n"+categories[j],x) else np.NaN).replace("\n","")

    
faa.to_csv("UAS_initialSplit1.csv")


In [744]:
# loop for the ASRS data to merge all files
asrsFolder = '\\ASRSdata'
asrs = pd.DataFrame()
for entry in os.scandir(directory + asrsFolder):
    temp = pd.read_csv(entry.path)
    asrs = asrs.append(temp)
asrs.to_csv("asrs_data.csv")